In [1]:
#Initial Setup

import numpy as np
import scipy.sparse as sp
from numpy.random import *
from multiprocessing import Pool
import pandas as pd
import pickle
import time
import os
import sys
import math
np.set_printoptions(threshold=sys.maxsize)

global maxInterference
maxInterference = 0.0

global trackTime
trackTime = 0

#Change parameters here
n = 20000 #Model size
p = 512 / 100000 #Chance a synapse exists
c = .65536 #Used in r approximation algorithm

d = (int) (p * n)
k = (int) (d * 32/512)


#r_approx = (int) ((n / 100000) * (pow(2, 16) * k / d))
r_approx = (int) (c * k / p)
M = []
rng = default_rng(seed=42)

print(n)
print(d)
print(k)
print(r_approx)

def create_sparse_gnp_graph() -> sp.csr_matrix:
        
    num_edges = rng.binomial(n * (n - 1) // 2, p)
    sources = rng.integers(0, n, num_edges * 2)
    targets = rng.integers(0, n, num_edges * 2)
    mask = sources != targets
    data = np.ones_like(sources[mask])
    adj_matrix = sp.csr_matrix((data, (targets[mask], sources[mask])), (n, n), dtype='bool')

    return adj_matrix

g = create_sparse_gnp_graph().toarray()

20000
102
6
768


In [ ]:
#Define SeqMem functions

#Checks interference between new memory A and existing memories M
def SeqMem_interference_check(A, i):
    global maxInterference
    this_max = 0
    for B_i in range(i):
        intersect = len(A & M[B_i])
        interference = intersect / len(M[B_i])
        
        if interference >= this_max:
            this_max = interference
        
    if this_max > maxInterference:
        maxInterference = this_max
    return this_max

#Generates new memory B using memory A
def SeqMem_one_step(A):
    fired_neighbors = np.zeros(n, dtype=int)
    for s_i in A:
        mask = g[s_i]
        fired_neighbors[mask] = fired_neighbors[mask]+1

    counts = np.zeros(k*6, dtype=int)
    for i in range(n):
        for j in range(fired_neighbors[i]+1):
            counts[j] = counts[j]+1

    k_s = np.searchsorted(-counts, -r_approx, side="left")
    if k_s > 0 and (k_s== len(counts) or math.fabs(r_approx - counts[k_s-1]) < math.fabs(r_approx - counts[k_s])):
        k_s = k_s-1
    
    B = set()
    for i in range(n):
        if fired_neighbors[i] >= k_s:
            B.add(i)

    #print("{} : {}".format(k_s, len(B)))

    return B, k_s

#Generates num new memories and appends the statistics to output file
def SeqMem_long_sequence(num):
    global maxInterference

    df = None

    if os.path.isfile('output.xlsx'):
        print("File found")
        df = pd.read_excel('output.xlsx')
        maxInterference = df.loc[len(df.index)-1]['Max Interference']
    else:
        print("Create new file")
        columns = ["Max Interference", "Interference", "Memory", "K"]
        df = pd.DataFrame(columns=columns)
        maxInterference = 0

    if len(M) == 0:
        M.append(set(rng.choice(np.arange(0,n-1), size=r_approx, replace=False)))
        new_row = [0, 0, r_approx, 0]
        df.loc[len(df.index)] = new_row


    memory_count = 1
    while True:
        #print("Memory {}".format(memory_count))

        A = M[-1]
        B, B_k = SeqMem_one_step(A)
        M.append(B)

        this_max = SeqMem_interference_check(B, len(M)-2)

        new_row = [maxInterference, this_max, len(B), B_k]
        df.loc[len(df.index)] = new_row

        if this_max >= 0.5 or memory_count >= num:
            df.to_excel('output.xlsx', index=False)
            return
        
        memory_count = memory_count + 1

#Generatles num new sequences of size L and appends the statistics to output file
def SeqMem_many_sequence(num, L):
    global maxInterference

    df = None

    if os.path.isfile('output.xlsx'):
        print("File found")
        df = pd.read_excel('output.xlsx')
        maxInterference = df.loc[len(df.index)-1]['Max Interference']
    else:
        print("Create new file")
        columns = ["Max Interference", "Interference"]
        for i in range(L):
            columns.append(i+1)
        df = pd.DataFrame(columns=columns)
        maxInterference = 0

    sequence_count = 1
    while True:
        #print("Sequence {}".format(sequence_count))

        A = set(rng.choice(np.arange(0,n-1), size=r_approx, replace=False))
        this_max = SeqMem_interference_check(A, len(M))
        M.append(A)

        S = [A]
        for i in range(L-1):
            B, B_k= SeqMem_one_step(S[-1])
            B_max = SeqMem_interference_check(B, len(M)-1)
            M.append(B)
            S.append(B)
            if(B_max > this_max):
                this_max = B_max

        if this_max > maxInterference:
            maxInterference = this_max

        new_row = [maxInterference, this_max]
        for i in range(len(S)):
            new_row.append(len(S[i]))
        df.loc[len(df.index)] = new_row

        if this_max >= 0.5 or sequence_count >= num:
            df.to_excel('output.xlsx', index=False)
            return
        
        sequence_count = sequence_count+1

In [11]:
#Saves current state to file. Use this when starting over and you want to reset the model and rng.

with open('M.pkl', 'wb') as f:
    pickle.dump(M, f)
np.save("rng.npy", rng.bit_generator.state)

In [12]:
#Main execution code. Loads everything from file, runs algorithm, and then saves everything to file.
#If loading doesn't work because you have created the files to load yet, run the section above this to save the empty memories and initial rng to file.

with open('M.pkl', 'rb') as f:
    M = pickle.load(f)
rng.bit_generator.__setstate__(np.load("rng.npy" ,allow_pickle='TRUE').item())


SeqMem_long_sequence(num=100)
#SeqMem_many_sequence(num=10, L=10)


with open('M.pkl', 'wb') as f:
    pickle.dump(M, f)
np.save("rng.npy", rng.bit_generator.state)

File found
Memory 1
8 : 764
Memory 2
8 : 907
Memory 3
9 : 920
Memory 4
9 : 975
Memory 5
10 : 623
Memory 6
7 : 887
Memory 7
9 : 791
Memory 8
8 : 1041
Memory 9
10 : 897
Memory 10
9 : 853
Memory 11
9 : 646
Memory 12
7 : 970
Memory 13
10 : 592
Memory 14
7 : 731
Memory 15
8 : 743
Memory 16
8 : 746
Memory 17
8 : 877
Memory 18
9 : 783
Memory 19
8 : 1028
Memory 20
10 : 877
Memory 21
9 : 801
Memory 22
9 : 484
Memory 23
6 : 814
Memory 24
9 : 526
Memory 25
6 : 1083
Memory 26
11 : 503
Memory 27
6 : 891
Memory 28
9 : 834
Memory 29
9 : 625
Memory 30
7 : 855
Memory 31
9 : 645
Memory 32
7 : 971
Memory 33
10 : 587
Memory 34
7 : 695
Memory 35
8 : 560
Memory 36
7 : 526
Memory 37
6 : 1092
Memory 38
11 : 570
Memory 39
7 : 580
Memory 40
7 : 639
Memory 41
7 : 987
Memory 42
10 : 645
Memory 43
7 : 982
Memory 44
10 : 620
Memory 45
7 : 810
Memory 46
9 : 490
Memory 47
6 : 861
Memory 48
9 : 700
Memory 49
8 : 612
Memory 50
7 : 778
Memory 51
8 : 953
Memory 52
10 : 535
Memory 53
7 : 432
Memory 54
6 : 542
Memory 55
7 

In [5]:
#Finds the relationship between incoming synapses and memory count.

with open('M.pkl', 'rb') as f:
    M = pickle.load(f)

memory_count = np.zeros(n, dtype=int)
for m in M:
    for v in m:
        memory_count[v] = memory_count[v] + 1

incoming = np.sum(g, axis=0)


columns = ["Num Memories", "Incoming"]
df = pd.DataFrame(columns=columns)

for i in range(n):
    #print(i)
    new_row = [memory_count[i], incoming[i]]
    df.loc[len(df.index)] = new_row

df.to_excel('neuron_info.xlsx', index=False)

In [6]:
#Demonstration of how you find the best k_s

A = set(rng.choice(np.arange(0,n-1), size=r_approx, replace=False))

fired_neighbors = np.zeros(n, dtype=int)
for s_i in A:
    mask = g[s_i]
    fired_neighbors[mask] = fired_neighbors[mask]+1

counts = np.zeros(k*6, dtype=int)
for i in range(n):
    for j in range(fired_neighbors[i]+1):
        counts[j] = counts[j]+1

print(counts)

[20000 19612 18089 15024 10992  7023  3949  1999   898   358   149    48
    19     5     1     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
